# 👟 Behaviors

Behaviors extend the appearance and interactivity of the [2D](#2D-Graph) and
[3D](./3D.ipynb) graphs. These can affect [nodes](#Node-Behaviors),
[links](#Link-Behaviors) or certain aspects of the graph itself.

In [1]:
if __name__ == "__main__" and "pyodide" in __import__("sys").modules:
    %pip install -q -r requirements.txt

In [2]:
import random

import ipywidgets as W
import traitlets as T

import ipyforcegraph.behaviors as B

In [3]:
with __import__("importnb").Notebook():
    import Utils as U
    from _index import make_a_simple_example

## 2D Graph

```{hint}
After rendering the cell below, select _Create New View For Output_ from the output's right-click menu to see more things added
```

In [4]:
if __name__ == "__main__":
    fg, box = make_a_simple_example()
    display(box)

## Graph Behaviors

### `ImageCapture`

The current image can be streamed back to the kernel.

In [5]:
def add_graph_image(fg, box):
    behavior = B.GraphImage()
    capturing = W.ToggleButton(description="capturing")
    frame_count = W.IntSlider()
    frames = W.HBox()
    ui = U.make_a_collapsible_picker(
        "graph: image",
        children={
            "capture": [
                W.VBox(
                    [
                        capturing,
                        frame_count,
                        frames,
                    ]
                )
            ]
        },
    )
    T.link((behavior, "capturing"), (capturing, "value"))
    T.link((behavior, "frame_count"), (frame_count, "value"))
    T.dlink((behavior, "frames"), (frames, "children"))
    box.behaviors = {**box.behaviors, "graph_image": behavior}
    box.graph_ui = {**box.graph_ui, "graph_image": ui}
    return fg, box

In [6]:
if __name__ == "__main__":
    add_graph_image(fg, box)

## Node Behaviors

### `NodeSelection`

The `NodeSelection` behavior allows for selecting one or more nodes from the browser, or
updating from the kernel.

In [7]:
def add_node_selection(fg, box):
    selection = B.NodeSelection()
    ui_selection = W.TagsInput(
        placeholder="select some nodes",
        allowed_tags=sorted(fg.source.nodes.id),
    )
    box.behaviors = {**box.behaviors, "node_selection": selection}
    box.node_ui = {**box.node_ui, "node_selection": ui_selection}
    T.link((selection, "selected"), (ui_selection, "value"))
    return fg, box

In [8]:
if __name__ == "__main__":
    add_node_selection(fg, box)

```{hint}
Note that the nodes changed colors. Click a node to select it, or use <kbd>ctrl</kbd> or <kbd>shift</kbd> to select multiple nodes.
```

The selection is handed back from the client, and can be used with other widgets.

### `NodeColors`

Node colors can be set based on a column value. By default, a column named `color` will
be used. Ensure `NodeColors` is in `behaviors` _after_ e.g. `NodeSelection`.

In [9]:
from ipyforcegraph.behaviors import NodeColors


def add_node_colors(fg=None, box=None, column_name="color"):
    if fg is None:
        fg, box = make_a_simple_example()

    colors = NodeColors(column_name=column_name)

    if column_name not in fg.source.nodes:
        U.make_random_color_series(fg, "nodes", column_name)
    ui_color_column = W.Dropdown(
        options=list(fg.source.nodes.columns), value=column_name
    )
    ui_color_template = W.Textarea(layout=dict(max_width="100%"))
    ui_color_template_enabled = W.Checkbox(description="enabled?")
    T.link((ui_color_column, "value"), (colors, "column_name"))
    T.dlink(
        (ui_color_template, "value"),
        (colors, "template"),
        lambda x: x if ui_color_template_enabled.value else "",
    )

    ui_colors = U.make_a_collapsible_picker(
        "node: color",
        {
            "off": [],
            "column": [ui_color_column],
            "template": [W.VBox([ui_color_template_enabled, ui_color_template])],
        },
    )
    box.behaviors = {**box.behaviors, "node_color": colors}
    box.node_ui = {**box.node_ui, "color": ui_colors}
    return fg, box

In [10]:
if __name__ == "__main__":
    add_node_colors(fg, box)

#### Color Templates

`NodeColors` support either a column name, and for the most part, calculating the values
derived for these as data frames is likely the best choice.

However, the`.template` traitlet, which take the form of
[nunjucks templates](https://mozilla.github.io/nunjucks/templating.html) allows for
calculating dynamic values on the client.

The syntax is intentionally very similar to
[jinja2](https://jinja.palletsprojects.com/en/3.1.x/templates).

Inside of a template, one can use:

- `node`
  - this will have all of the named columns available to it
- `graphData`
  - `nodes`
  - `links`
    - `source` and `target` as realized nodes

With these, and basic template tools, one can generate all kinds of interesting effects.
For the example data above, try these color templates:

- color by group
  > ```python
  > {{ ["red", "yellow", "blue", "orange", "purple", "magenta"][node.group] }}
  > ```
- color by out-degree
  > ```python
  > {% set n = 0 %}
  > {% for link in graphData.links %}
  >   {% if link.source.id == node.id %}{% set n = n + 1 %}{% endif %}
  > {% endfor %}
  > {% set c = 256 * (7-n) / 7 %}
  > rgb({{ c }},0,0)
  > ```

### `NodeLabels`

Node labels can be revealed when hovering over the node. By default the node's `id`
column will be used.

In [11]:
from ipyforcegraph.behaviors import NodeLabels


def add_node_labels(fg, box, column_name="id"):
    labels = NodeLabels()
    ui_label_column = W.Dropdown(
        options=list(fg.source.nodes.columns), value=column_name
    )
    ui_label_template = W.Textarea()
    ui_label_template_enabled = W.Checkbox(description="enabled?")
    T.link((ui_label_column, "value"), (labels, "column_name"))
    T.dlink(
        (ui_label_template, "value"),
        (labels, "template"),
        lambda x: x if ui_label_template_enabled.value else "",
    )
    ui_labels = U.make_a_collapsible_picker(
        "node: label",
        {
            "off": [],
            "column": [ui_label_column],
            "template": [W.VBox([ui_label_template_enabled, ui_label_template])],
        },
    )
    box.behaviors = {**box.behaviors, "node_label": labels}
    box.node_ui = {**box.node_ui, "label": ui_labels}
    return fg, box

In [12]:
if __name__ == "__main__":
    add_node_labels(fg, box)

#### Label templates

Like `NodeColors`, `NodeLabels` also accepts [templates](#color-templates). The
resulting value may be plain strings or HTML.

Here are some examples, again for the example data:

- just a header
  > ```html
  > <h1>{{ node.id }}</h1>
  > ```
- a table
  > ```html
  > <table>
  >  <tr><th>id</th><th>group</th></td>
  >  {% for link in graphData.links %}
  >  {% if link.source.id == node.id %}
  >  <tr><td>{{ link.target.id }}</td><td>{{ link.target.group }}</td>
  >  {% endif %}
  >  {% endfor %}
  > </table>
  > ```

## Link Behaviors

### `LinkColors`

Link colors can also be configured by column name or template.

In [13]:
add_link_colors = U.make_link_behavior_with_ui(
    B.LinkColors, "link: color", "color", True
)

In [14]:
if __name__ == "__main__":
    add_link_colors(fg, box)

### `LinkLabels`

Link labels are revealed when hovered, and accept `column_name` or `template`.

In [15]:
add_link_labels = U.make_link_behavior_with_ui(B.LinkLabels, "link: label", "value")

if __name__ == "__main__":
    add_link_labels(fg, box)

### `LinkDirectionalArrowColor`

In [16]:
add_link_directional_arrow_color = U.make_link_behavior_with_ui(
    B.LinkDirectionalArrowColor, "link: directional arrow color", "color", is_color=True
)

if __name__ == "__main__":
    add_link_directional_arrow_color(fg, box)

### `LinkDirectionalArrowLength`

In [17]:
add_link_directional_arrow_length = U.make_link_behavior_with_ui(
    B.LinkDirectionalArrowLength, "link: directional arrow length", "value"
)

if __name__ == "__main__":
    add_link_directional_arrow_length(fg, box)

### `LinkDirectionalArrowRelPos`

In [18]:
add_link_directional_arrow_rel_pos = U.make_link_behavior_with_ui(
    B.LinkDirectionalArrowRelPos, "link: directional arrow relative position", "value"
)

if __name__ == "__main__":
    add_link_directional_arrow_rel_pos(fg, box)

### `LinkDirectionalParticleColor`

In [19]:
add_link_directional_particle_color = U.make_link_behavior_with_ui(
    B.LinkDirectionalParticleColor,
    "link: directional particle color",
    "color",
    is_color=True,
)

if __name__ == "__main__":
    add_link_directional_particle_color(fg, box)

### `LinkDirectionalParticleSpeed`

In [20]:
add_link_directional_particle_speed = U.make_link_behavior_with_ui(
    B.LinkDirectionalParticleSpeed, "link: directional particle speed", "value"
)

if __name__ == "__main__":
    add_link_directional_particle_speed(fg, box)

### `LinkDirectionalParticleWidth`

In [21]:
add_link_directional_particle_width = U.make_link_behavior_with_ui(
    B.LinkDirectionalParticleWidth, "link: directional particle width", "value"
)

if __name__ == "__main__":
    add_link_directional_particle_width(fg, box)

### `LinkDirectionalParticles`

In [22]:
add_link_directional_particles = U.make_link_behavior_with_ui(
    B.LinkDirectionalParticles, "link: directional particles", "value"
)

if __name__ == "__main__":
    add_link_directional_particles(fg, box)